# Load Prep

Using this notebook to test the structure and load of our data into the Postgres tables.

## Imports

In [22]:
import pandas as pd
import numpy as np
import psycopg2
import os
import datetime
import time
from dotenv import load_dotenv

## Get into format for load

My process for the test will be to load in one of the sample CSVs into a dataframe and loop through the values to get INSERT statements built. The actual workflow will be different start from the API instead of the CSV.

I think I'll test on one with a date value to make sure I get a chance to see how that preps. DIM_DATE would be a good one.

Plus this one isn't dependet on any others, so I'll try it.

### Load DataFrame

In [2]:
#load
df_date = pd.read_csv('../data/Exports/dim_date.csv')

#convert to date format
df_date['full_date'] = pd.to_datetime(df_date['full_date'])

#drop unnamed
df_date = df_date.drop(columns='Unnamed: 0')

df_date.head()

,date_id,full_date,day_of_week,day_of_week_num,day_of_month,month,quarter,year
0,20220101,2022-01-01,Saturday,7,1,1,1,2022
1,20220102,2022-01-02,Sunday,1,2,1,1,2022
2,20220103,2022-01-03,Monday,2,3,1,1,2022
3,20220104,2022-01-04,Tuesday,3,4,1,1,2022
4,20220105,2022-01-05,Wednesday,4,5,1,1,2022


### Create INSERT Statements

I think there is a pandas function 'to_sql', but I want to be able to see under the hood a little more. So I want to itterate through the records to create SQL insert statements that can then be run using sqlalchemy or psycopg2.

In [3]:
#set data table name
schema_table = 'nfl.dim_date'

In [4]:
#save column names to a list
column_names = df_date.columns.tolist()

In [5]:
#create empty list to store insert statements
insert_statements = []

for index, row in df_date.iterrows():
    values = ', '.join([f"'{val}'" if isinstance(val, (str, datetime.datetime)) and not pd.isna(val) else 'NULL' if pd.isna(val) else str(val) for val in row])
    insert_statement = f"INSERT INTO {schema_table} ({', '.join(column_names)}) VALUES ({values});"
    insert_statements.append(insert_statement)

In [6]:
for sql in insert_statements[:3]:
    print(sql)

INSERT INTO nfl.dim_date (date_id, full_date, day_of_week, day_of_week_num, day_of_month, month, quarter, year) VALUES (20220101, '2022-01-01 00:00:00', 'Saturday', 7, 1, 1, 1, 2022);
INSERT INTO nfl.dim_date (date_id, full_date, day_of_week, day_of_week_num, day_of_month, month, quarter, year) VALUES (20220102, '2022-01-02 00:00:00', 'Sunday', 1, 2, 1, 1, 2022);
INSERT INTO nfl.dim_date (date_id, full_date, day_of_week, day_of_week_num, day_of_month, month, quarter, year) VALUES (20220103, '2022-01-03 00:00:00', 'Monday', 2, 3, 1, 1, 2022);


Looks good! Now for the actual load test.

### Test Load to PostgreSQL Database

In [7]:
#setup connection variables
# Load .env to get PostgreSQL user login info
load_dotenv()

# set postgres access variables
db_host = 'localhost'
db_user = os.getenv('psql_username')
db_password = os.getenv('psql_password')
db_name = 'team_flow'

#put connection details into params variable
db_params = {
    'host': '{host}'.format(host=db_host),
    'database': '{database}'.format(database=db_name),  
    'user': '{user}'.format(user=db_user),
    'password': '{password}'.format(password=db_password)
}

In [9]:
len(insert_statements)

730

In [8]:
#try load
try:
    #setup connection and cursor
    conn = psycopg2.connect(**db_params)
    cursor = conn.cursor()

    #insert data
    for sql in insert_statements:
        cursor.execute(sql)

    #commit changes
    conn.commit()

    count = len(insert_statements)
    print(count, "records inserted successfully into {schema_table} table".format(schema_table=schema_table))

except (Exception, psycopg2.Error) as error:
    print(f"Error: {error}")

finally:
    # Close the cursor and connection
    if cursor:
        cursor.close()
    if conn:
        conn.close()

1 records inserted successfully into nfl.dim_date table


I made an update there to not just say 1 record updated, but the actual count.

But now with that let's move on to the game data and betting to get a nice chain of dependent loads.

### Load Dataframe

In [16]:
#load
df_game = pd.read_csv('../data/Exports/dim_game.csv')

#drop unnamed
df_game = df_game.drop(columns='Unnamed: 0')

df_game.head()

,game_id,game_date_id,game_type_id,home_team_id,away_team_id,game_start_time,game_location,game_arena,is_neutral_site_flag,espn_link,cbs_link
0,20220911_KC@ARI,20220911,2,1,16,4:25p,"Glendale, AZ",NaN,0,https://www.espn.com/nfl/boxscore/_/gameId/401...,https://www.cbssports.com/nfl/gametracker/boxs...
1,20220918_ARI@LV,20220918,2,17,1,4:25p,"Las Vegas, NV",NaN,0,https://www.espn.com/nfl/boxscore/_/gameId/401...,https://www.cbssports.com/nfl/gametracker/boxs...


### Create INSERT Statements

In [17]:
#set data table name
schema_table = 'nfl.dim_game'

In [18]:
#save column names to a list
column_names = df_game.columns.tolist()

In [26]:
column_names

['game_id',
 'game_date_id',
 'game_type_id',
 'home_team_id',
 'away_team_id',
 'game_start_time',
 'game_location',
 'game_arena',
 'is_neutral_site_flag',
 'espn_link',
 'cbs_link']

In [23]:
#create empty list to store insert statements
insert_statements = []

for index, row in df_game.iterrows():
    values = ', '.join([f"'{val}'" if isinstance(val, (str, datetime.datetime)) and not pd.isna(val) else 'NULL' if pd.isna(val) else str(val) for val in row])
    insert_statement = f"INSERT INTO {schema_table} ({', '.join(column_names)}) VALUES ({values});"
    insert_statements.append(insert_statement)

In [24]:
for sql in insert_statements[:3]:
    print(sql)

INSERT INTO nfl.dim_game (game_id, game_date_id, game_type_id, home_team_id, away_team_id, game_start_time, game_location, game_arena, is_neutral_site_flag, espn_link, cbs_link) VALUES ('20220911_KC@ARI', 20220911, 2, 1, 16, '4:25p', 'Glendale, AZ', NULL, 0, 'https://www.espn.com/nfl/boxscore/_/gameId/401437653', 'https://www.cbssports.com/nfl/gametracker/boxscore/NFL_20220911_KC@ARI');
INSERT INTO nfl.dim_game (game_id, game_date_id, game_type_id, home_team_id, away_team_id, game_start_time, game_location, game_arena, is_neutral_site_flag, espn_link, cbs_link) VALUES ('20220918_ARI@LV', 20220918, 2, 17, 1, '4:25p', 'Las Vegas, NV', NULL, 0, 'https://www.espn.com/nfl/boxscore/_/gameId/401437642', 'https://www.cbssports.com/nfl/gametracker/boxscore/NFL_20220918_ARI@LV');


### Load to Database

In [25]:
#try load
try:
    #setup connection and cursor
    conn = psycopg2.connect(**db_params)
    cursor = conn.cursor()

    #insert data
    for sql in insert_statements:
        cursor.execute(sql)

    #commit changes
    conn.commit()

    count = cursor.rowcount
    print(count, "records inserted successfully into {schema.table} table".format(schema_table=schema_table))

except (Exception, psycopg2.Error) as error:
    print(f"Error: {error}")

finally:
    # Close the cursor and connection
    if cursor:
        cursor.close()
    if conn:
        conn.close()

Error: insert or update on table "dim_game" violates foreign key constraint "fk_dim_game_game_type"
DETAIL:  Key (game_type_id)=(2) is not present in table "dim_game_type".



In [2]:
#read in fct_betting sample CSV
df = pd.read_csv('../data/Exports/fct_betting.csv')

df['last_update'] = pd.to_datetime(df['last_update'])

df.head()

,Unnamed: 0,game_id,sportsbook_id,last_update,total_over_under,over_odds,under_odds,home_spread,home_odds,away_spread,away_odds,home_ml_odds,away_ml_odds
0,0,20230924_DAL@ARI,1,2023-09-24 13:26:08.483308800,43.0,-110,-110,12.5,-105,-12.5,-115,525,-750
1,1,20230924_DAL@ARI,2,2023-09-24 13:26:08.483308800,43.0,-110,-110,13.0,-110,-13.0,-110,525,-750
2,2,20230924_DAL@ARI,3,2023-09-24 13:26:08.483308800,43.5,-105,-115,12.5,-110,-12.5,-110,520,-720
3,3,20230924_DAL@ARI,5,2023-09-24 13:26:08.483308800,43.0,-112,-109,13.0,-110,-13.0,-110,500,-770
4,4,20230924_DAL@ARI,6,2023-09-24 13:26:08.483308800,43.5,-110,-110,12.5,-110,-12.5,-110,550,-800


In [3]:
#drop unnamed column
df = df.drop(columns='Unnamed: 0')

df.head()

,game_id,sportsbook_id,last_update,total_over_under,over_odds,under_odds,home_spread,home_odds,away_spread,away_odds,home_ml_odds,away_ml_odds
0,20230924_DAL@ARI,1,2023-09-24 13:26:08.483308800,43.0,-110,-110,12.5,-105,-12.5,-115,525,-750
1,20230924_DAL@ARI,2,2023-09-24 13:26:08.483308800,43.0,-110,-110,13.0,-110,-13.0,-110,525,-750
2,20230924_DAL@ARI,3,2023-09-24 13:26:08.483308800,43.5,-105,-115,12.5,-110,-12.5,-110,520,-720
3,20230924_DAL@ARI,5,2023-09-24 13:26:08.483308800,43.0,-112,-109,13.0,-110,-13.0,-110,500,-770
4,20230924_DAL@ARI,6,2023-09-24 13:26:08.483308800,43.5,-110,-110,12.5,-110,-12.5,-110,550,-800


### Create INSERT Statements

I think there is a pandas function 'to_sql', but I want to be able to see under the hood a little more. So I want to itterate through the records to create SQL insert statements that can then be run using sqlalchemy or psycopg2.

In [4]:
#set data table name
schema_table = 'nfl.fct_betting'

In [5]:
#save column names to a list
column_names = df.columns.tolist()

In [6]:
#create empty list to store insert statements
insert_statements = []

for index, row in df.iterrows():
    values = (', '.join([f"'{val}'" if isinstance(val, (str, datetime.datetime)) else str(val) for val in row]))
    insert_statement = f"INSERT INTO {schema_table} ({', '.join(column_names)}) VALUES ({values});"
    insert_statements.append(insert_statement)

In [7]:
for sql in insert_statements:
    print(sql)

INSERT INTO nfl.fct_betting (game_id, sportsbook_id, last_update, total_over_under, over_odds, under_odds, home_spread, home_odds, away_spread, away_odds, home_ml_odds, away_ml_odds) VALUES ('20230924_DAL@ARI', 1, '2023-09-24 13:26:08.483308800', 43.0, -110, -110, 12.5, -105, -12.5, -115, 525, -750);
INSERT INTO nfl.fct_betting (game_id, sportsbook_id, last_update, total_over_under, over_odds, under_odds, home_spread, home_odds, away_spread, away_odds, home_ml_odds, away_ml_odds) VALUES ('20230924_DAL@ARI', 2, '2023-09-24 13:26:08.483308800', 43.0, -110, -110, 13.0, -110, -13.0, -110, 525, -750);
INSERT INTO nfl.fct_betting (game_id, sportsbook_id, last_update, total_over_under, over_odds, under_odds, home_spread, home_odds, away_spread, away_odds, home_ml_odds, away_ml_odds) VALUES ('20230924_DAL@ARI', 3, '2023-09-24 13:26:08.483308800', 43.5, -105, -115, 12.5, -110, -12.5, -110, 520, -720);
INSERT INTO nfl.fct_betting (game_id, sportsbook_id, last_update, total_over_under, over_odds,

Looks good! Now for the actual load test.

### Test Load to PostgreSQL Database

In [8]:
#setup connection variables
# Load .env to get PostgreSQL user login info
load_dotenv()

# set postgres access variables
db_host = 'localhost'
db_user = os.getenv('psql_username')
db_password = os.getenv('psql_password')
db_name = 'team_flow'

#put connection details into params variable
db_params = {
    'host': '{host}'.format(host=db_host),
    'database': '{database}'.format(database=db_name),  
    'user': '{user}'.format(user=db_user),
    'password': '{password}'.format(password=db_password)
}

In [9]:
#try load
try:
    #setup connection and cursor
    conn = psycopg2.connect(**db_params)
    cursor = conn.cursor()

    #insert data
    for sql in insert_statements:
        cursor.execute(sql)

    #commit changes
    conn.commit()

    count = cursor.rowcount
    print(count, "records inserted successfully into {schema.table} table".format(schema_table=schema_table))

except (Exception, psycopg2.Error) as error:
    print(f"Error: {error}")

finally:
    # Close the cursor and connection
    if cursor:
        cursor.close()
    if conn:
        conn.close()

Error: insert or update on table "fct_betting" violates foreign key constraint "fk_fct_betting_game"
DETAIL:  Key (game_id)=(20230924_DAL@ARI) is not present in table "dim_game".

